<br/>
<br/>

# Part 4: Ensembling Is All You Need -- Blending To Achieve Higher Score 
<br/>
<br/>
Ensemble is a very powerful machine learning techique, we did't spend a lot of time to do this, but simply blending all of the model can give us great boost.

What's more variety is very important.

We have 5 good models:

Bert Large------------0.9430<br/>
Bert Base 1----------0.9415<br/>
Bert Base 2----------0.9413<br/>
GTP2------------------0.9387<br/>
Ensemble LSTM----0.9369<br/>

Although LSTM and GTP2 have relatively low score, but blending them to gether with Berts can give a great boost.

What we did is:<br/>
**Finall =  0.350*Bert_Large + 0.175*Bert_Base_1 + 0.175*Bert_Base_2 + 0.170*GPT2 + 0.130*LSTM** <br/>
which improve the single best **0.9430 --> 0.9446**

It's worth to mention that the **weight average of the score is 0.9409**, but simply blending the models can **lower the variance and gives us 0.9446**.

# Blending Pipline
<br/>
Here's a pipline that load all models and make predictions. Nothing special.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
from pprint import pprint
import pandas as pd
import os
import time
import pickle
import gc
import random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
device = torch.device('cuda')
import pickle

Using TensorFlow backend.


# Ensemble  LSTM Prediction

In [2]:
CRAWL_EMBEDDING_PATH = '../input/lstm-word-embedding/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/lstm-word-embedding/glove.840B.300d.pkl'

In [3]:
NUM_MODELS = 2
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 300

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

In [4]:
def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        unknown_words.append(word)
    return embedding_matrix, unknown_words

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

In [6]:
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

In [7]:
symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

In [8]:
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    small_caps_mapping = { 
    "ᴀ": "a", "ʙ": "b", "ᴄ": "c", "ᴅ": "d", "ᴇ": "e", "ғ": "f", "ɢ": "g", "ʜ": "h", "ɪ": "i", 
    "ᴊ": "j", "ᴋ": "k", "ʟ": "l", "ᴍ": "m", "ɴ": "n", "ᴏ": "o", "ᴘ": "p", "ǫ": "q", "ʀ": "r", 
    "s": "s", "ᴛ": "t", "ᴜ": "u", "ᴠ": "v", "ᴡ": "w", "x": "x", "ʏ": "y", "ᴢ": "z"}
    contraction_mapping = {
    "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  
    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": 
    "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", 
    "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", 
    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", 
    "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's":"this is","that'd": "that would", 
    "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", 
    "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", 
    "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", 
    "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", 
    "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", 
    "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have",
    "trump's": "trump is", "obama's": "obama is", "canada's": "canada is", "today's": "today is"}
    specail_signs = { "…": "...", "₂": "2"}
    specials = ["’", "‘", "´", "`"]

In [9]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()


isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

In [10]:
x_test = test['comment_text'].apply(lambda x:preprocess(x))
max_features = 400000

In [11]:
tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False)

In [12]:
tokenizer.fit_on_texts(list(x_test))

crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

max_features = max_features or len(tokenizer.word_index) + 1
max_features

embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix.shape

del crawl_matrix
del glove_matrix
gc.collect()

n unknown words (crawl):  14055
n unknown words (glove):  14375


0

In [13]:
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, x, lengths=None):
        h_embedding = self.embedding(x.long())
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [14]:
maxlen = 230

In [15]:
batch_size = 512
tokenized_test = tokenizer.texts_to_sequences(x_test)
#test_lengths = torch.from_numpy(np.array([len(x) for x in tokenized_test]))
x_test_padded = torch.from_numpy(sequence.pad_sequences(tokenized_test, maxlen=maxlen))
test_dataset = data.TensorDataset(x_test_padded)

In [16]:
def make_prediction(test,model_path,batch_size=256):
    model = NeuralNet(embedding_matrix, 6)
    temp_dict = torch.load(model_path)
    temp_dict['embedding.weight'] = torch.tensor(embedding_matrix)
    model.load_state_dict(temp_dict)
    model.cuda()
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    test_preds = np.zeros(len(test))    
    step = 0
    total_step = len(test_loader)
    for i, (x_batch,) in enumerate(test_loader):
        if step %100 == 0:
            print('step:',step,'total_step:',total_step)
        step += 1
        pred = model(x_batch.to(device))
        test_preds[i * batch_size:(i + 1) * batch_size] = pred[:, 0].detach().cpu().squeeze().numpy()

    test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()
    return test_pred

In [17]:
model_path_epoch4 = ['../input/4lstm-model/model1_lstm_epoch4.bin',
                    '../input/4lstm-model/model2_lstm_epoch4.bin',
                    '../input/4lstm-model/model3_lstm_epoch4.bin',
                    '../input/4lstm-model/model4_lstm_epoch4.bin']
model_path_epoch3 = ['../input/4lstm-model/model1_lstm_epoch3.bin',
                    '../input/4lstm-model/model2_lstm_epoch3.bin',
                    '../input/4lstm-model/model3_lstm_epoch3.bin',
                    '../input/4lstm-model/model4_lstm_epoch3.bin']
model_path_epoch2 = ['../input/4lstm-model/model1_lstm_epoch2.bin',
                    '../input/4lstm-model/model2_lstm_epoch2.bin',
                    '../input/4lstm-model/model3_lstm_epoch2.bin',
                    '../input/4lstm-model/model4_lstm_epoch2.bin']

In [18]:
epoch4_pred = []
for model_path in model_path_epoch4:
    temp_test_pred = make_prediction(test_dataset, model_path, batch_size=512)
    epoch4_pred.append(temp_test_pred)

step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191


In [19]:
epoch3_pred = []
for model_path in model_path_epoch3:
    temp_test_pred = make_prediction(test_dataset, model_path, batch_size=512)
    epoch3_pred.append(temp_test_pred)

step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191


In [20]:
epoch2_pred = []
for model_path in model_path_epoch2:
    temp_test_pred = make_prediction(test_dataset, model_path, batch_size=512)
    epoch2_pred.append(temp_test_pred)

step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191
step: 0 total_step: 191
step: 100 total_step: 191


In [21]:
epoch4_average = np.zeros(len(x_test))
for pred in epoch4_pred:
    epoch4_average += pred/4
    
epoch3_average = np.zeros(len(x_test))
for pred in epoch3_pred:
    epoch3_average += pred/4
    
epoch2_average = np.zeros(len(x_test))
for pred in epoch2_pred:
    epoch2_average += pred/4

In [22]:
checkpoint_weights = [6,8,4]
lstm_pred = np.average([epoch4_average,epoch3_average,epoch2_average], weights=checkpoint_weights, axis=0)  

In [23]:
submission_lstm = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction': lstm_pred
})

#submission_lstm.to_csv('submission.csv', index=False)

In [24]:
del embedding_matrix
gc.collect()

0

# GTP2 Prediction 

In [27]:
# the kernel environment don't have the last version of pytorch bert, so we need to install by ourselves.
!pip install ../input/finalpytorchgpt/our-gpt2-pytorch/pytorch-pretrained-BERT-master/

Processing /kaggle/input/finalpytorchgpt/our-gpt2-pytorch/pytorch-pretrained-BERT-master
  Stored in directory: /tmp/.cache/pip/wheels/62/41/7f/a49d8fa715ffad54d21ed24028d5568b616321f3355a305dd1
Successfully built pytorch-pretrained-bert
ERROR: allennlp 0.8.4 requires awscli>=1.11.91, which is not installed.
ERROR: allennlp 0.8.4 requires flaky, which is not installed.
ERROR: allennlp 0.8.4 requires responses>=0.7, which is not installed.
  Found existing installation: pytorch-pretrained-bert 0.6.2
    Uninstalling pytorch-pretrained-bert-0.6.2:
      Successfully uninstalled pytorch-pretrained-bert-0.6.2


In [28]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm,tqdm_notebook
import torch.nn.functional as F
import os
import warnings
# from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam

from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Config
from pytorch_pretrained_bert.modeling_gpt2 import *
warnings.filterwarnings(action='once')
device = torch.device('cuda')

In [29]:
## Testset Collator for triming long sequences --- put it in Pytorch DataLoader
class TestCollator():
    def __init__(self, sequence_index,length_index,choose_length = lambda x: x.max()):
        self.choose_length = choose_length
        self.sequence_index = sequence_index
        self.length_index = length_index

    def __call__(self, batch):
        batch = [torch.stack(x) for x in zip(*batch)]
        
        sequences = batch[self.sequence_index]
        lengths = batch[self.length_index]
        length = self.choose_length(lengths)
        padded_sequences = sequences[:, 0:length]     
        batch[self.sequence_index] = padded_sequences
        
        return batch[self.sequence_index]

## GTP2 Tokenization

In [30]:
def convert_lines(example):
    '''
    example shoulbe a data 
    '''
    all_tokens = []
    all_length = []
    longer = 0
    for text in tqdm_notebook(example):
        # print(text)
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>MAX_SEQUENCE_LENGTH:
            tokens_a = tokens_a[:MAX_SEQUENCE_LENGTH]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(tokens_a)
        one_token_length = len(one_token)
        one_token += [0]*(MAX_SEQUENCE_LENGTH - len(tokens_a))
        all_tokens.append(one_token)
        all_length.append(one_token_length)
    print(longer)
    return np.array(all_tokens),np.array(all_length) # Note that including Sequence Length Here

In [31]:
MAX_SEQUENCE_LENGTH = 300
SEED = 1234
BATCH_SIZE = 128
# BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
tokenizer = GPT2Tokenizer.from_pretrained('../input/gpt2-models/')

/opt/conda/lib/python3.6/site-packages/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
/opt/conda/lib/python3.6/site-packages/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]


In [32]:
test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test,seq_length = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"))


32


## GPT2 0.9388 Prediction

In [33]:
config = GPT2Config()

In [34]:
class GPT2ClassificationHeadModel(GPT2PreTrainedModel):

    def __init__(self, config, clf_dropout=0.4, n_class=7):
        super(GPT2ClassificationHeadModel, self).__init__(config)
        self.transformer = GPT2Model(config)
        self.dropout = nn.Dropout(clf_dropout)
        self.linear1 = nn.Linear(config.n_embd * 2, config.n_embd)
        self.linear = nn.Linear(config.n_embd, n_class)
        self.linear2 = nn.Linear(config.n_embd, config.n_embd)
        self.out = nn.Linear(config.n_embd, 1)
        # self.bn1 = nn.BatchNorm1d(config.n_embd, affine = False)
        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.normal_(self.linear.bias, 0)
        
        self.apply(self.init_weights)


    def forward(self, input_ids, position_ids=None, token_type_ids=None, lm_labels=None, past=None):
        hidden_states, presents = self.transformer(input_ids, position_ids, token_type_ids, past)
        avg_pool = torch.mean(hidden_states, 1)
        max_pool, _ = torch.max(hidden_states, 1)
        
        h_conc = torch.cat((avg_pool, max_pool), 1)
        
        hidden = self.dropout(F.relu(self.linear1(h_conc)))
        hidden1 = self.dropout(F.relu(self.linear2(hidden)))
        
        h = hidden + hidden1
        
        
        output = self.out(self.dropout(h))
        aux = self.linear(self.dropout(h))
        
        logits = torch.cat([output, aux], 1)
        
        return logits

In [35]:
model = GPT2ClassificationHeadModel(config, 0.3,7)
model = torch.nn.DataParallel(model)

In [36]:
os.listdir("../input/gpt-dense-0622")

['GPT2_full_0622.bin']

In [37]:
model.load_state_dict(torch.load("../input/gpt-dense-0622/GPT2_full_0622.bin"))

model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

DataParallel(
  (module): GPT2ClassificationHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (h): ModuleList(
        (0): Block(
          (ln_1): BertLayerNorm()
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
          )
          (ln_2): BertLayerNorm()
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
          )
        )
        (1): Block(
          (ln_1): BertLayerNorm()
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
          )
          (ln_2): BertLayerNorm()
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
          )
        )
        (2): Block(
          (ln_1): BertLayerNorm()
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
          )
          (ln_2): BertLayerNorm()
          (mlp): MLP(
            (c_fc): 

In [38]:
test_collator = TestCollator(0,1)
seq_length_torch = torch.tensor(seq_length,dtype=torch.long)
X_test_torch =torch.tensor(X_test, dtype=torch.long)
test = torch.utils.data.TensorDataset(X_test_torch,seq_length_torch)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False,collate_fn=test_collator)
test_preds = np.zeros((len(X_test)))

In [39]:
tk0 = tqdm_notebook(test_loader)
for i, x_batch in enumerate(tk0):
    pred = model(x_batch.to(device))
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [40]:
submission_gpt2_1 = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred
})
#submission_gpt2_1.to_csv('submission.csv', index=False)

# Bert Large

In [42]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm,tqdm_notebook
import os
import warnings
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig

warnings.filterwarnings(action='once')
device = torch.device('cuda')

In [43]:
# Add the Bart Pytorch repo to the PATH
# using files from: https://github.com/huggingface/pytorch-pretrained-BERT
# package_dir_a = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
# sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

In [44]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    all_length = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])
        one_token_length = len(one_token)
        one_token = one_token + [0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
        all_length.append(one_token_length)
    print(longer)
    return np.array(all_tokens),np.array(all_length) # Note that including Sequence Length Here

## Convert Tensorflow Weight to Pytorch Weight

In [45]:
MAX_SEQUENCE_LENGTH = 320 ### 320 is enough
SEED = 1234 
BATCH_SIZE = 64
BERT_MODEL_PATH = '../input/bert-pretrained-model-uncased/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'
Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification" 
Input_dir = "../input"  ###############
WORK_DIR = "../working/" ############



#bert_config = BertConfig('../input/prob-target333/bert_config.json')
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

##  Tokenization

In [46]:

test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test,seq_length = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)


6


In [47]:
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /kaggle/input/bert-pretrained-model-uncased/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [1024]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 1024]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 1024]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [1024]
Loading TF wei

'../working/bert_config.json'

## Build up Model & Makes Prediction

In [48]:
BERT_OUT = 128
DENSE_HIDDEN_UNITS = 128

In [49]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        #self.bert_layer = BertForSequenceClassification(bert_config,num_labels = BERT_OUT)
        self.bert_layer = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels = BERT_OUT)
        self.linear_out = nn.Linear(BERT_OUT, 1)
        self.linear_aux_out = nn.Linear(BERT_OUT, 7)
        #self.identity_out = nn.Linear(BERT_OUT, 9)
        self.drop_out_layer = nn.Dropout(p=0.1)        
        
    def forward(self, x, attention_mask=None, labels=None):
        bert_out = self.bert_layer(x, attention_mask=attention_mask, labels=labels)
        drop_out_layer_out = self.drop_out_layer(bert_out)
        result = self.linear_out(drop_out_layer_out)
        aux_result = self.linear_aux_out(bert_out)
        #identity_result = self.identity_out(bert_out)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [50]:
model = NeuralNet()
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load("../input/bert-large/bert_large_0.bin"))
model.cuda()
for param in model.parameters():
    param.requires_grad = False
model.eval()

DataParallel(
  (module): NeuralNet(
    (bert_layer): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 1024, padding_idx=0)
          (position_embeddings): Embedding(512, 1024)
          (token_type_embeddings): Embedding(2, 1024)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=1024, out_features=1024, bias=True)
                  (key): Linear(in_features=1024, out_features=1024, bias=True)
                  (value): Linear(in_features=1024, out_features=1024, bias=True)
                  (dropout): Dropout(p=0.1)
                )
                (output): BertSelfOutput(
                  (dense): Linear(in_features=1024, out_features=10

In [51]:
test_collator = TestCollator(0,1)
seq_length_torch = torch.tensor(seq_length,dtype=torch.long)
X_test_torch =torch.tensor(X_test, dtype=torch.long)
test = torch.utils.data.TensorDataset(X_test_torch,seq_length_torch)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False,collate_fn=test_collator)
test_preds = np.zeros((len(X_test)))

In [52]:
tk0 = tqdm_notebook(test_loader)
for i, x_batch in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [53]:
submission_bert_large = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred
})
#submission_bert_large.to_csv('submission_bert_large.csv', index=False)

In [54]:
submission_bert_large.head()

,id,prediction
0,7097320,0.000141
1,7097321,0.016696
2,7097322,0.061666
3,7097323,0.003367
4,7097324,0.001364


# Bert Base


## Tokenization For Bert

In [55]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    all_length = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])
        one_token_length = len(one_token)
        one_token = one_token + [0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
        all_length.append(one_token_length)
    print(longer)
    return np.array(all_tokens),np.array(all_length) # Note that including Sequence Length Here

In [56]:
MAX_SEQUENCE_LENGTH = 320 ### 320 is enough
SEED = 1234 
BATCH_SIZE = 64
BERT_MODEL_PATH = '../input/bert-pretrained-model-uncased/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification" 
Input_dir = "../input"  ###############
WORK_DIR = "../working/" ############



#bert_config = BertConfig('../input/prob-target333/bert_config.json')
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

## Convert Tensorflow Weight to Pytorch Weight

In [57]:
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /kaggle/input/bert-pretrained-model-uncased/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/

'../working/bert_config.json'

## BertModel_1 0.94135LB 0.94185CV

In [58]:
BERT_OUT = 128
DENSE_HIDDEN_UNITS = 128

In [59]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        #self.bert_layer = BertForSequenceClassification(bert_config,num_labels = BERT_OUT)
        self.bert_layer = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels = BERT_OUT)
        self.linear_out = nn.Linear(BERT_OUT, 1)
        self.linear_aux_out = nn.Linear(BERT_OUT, 8)
        #self.identity_out = nn.Linear(BERT_OUT, 9)
        self.drop_out_layer = nn.Dropout(p=0.1)        
        
    def forward(self, x, attention_mask=None, labels=None):
        bert_out = self.bert_layer(x, attention_mask=attention_mask, labels=labels)
        drop_out_layer_out = self.drop_out_layer(bert_out)
        result = self.linear_out(drop_out_layer_out)
        aux_result = self.linear_aux_out(bert_out)
        #identity_result = self.identity_out(bert_out)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [60]:
model = NeuralNet()
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load("../input/prob-target333/batch128_probtarget2_1.bin"))
model.cuda()
for param in model.parameters():
    param.requires_grad = False
model.eval()

DataParallel(
  (module): NeuralNet(
    (bert_layer): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1)
                )
                (output): BertSelfOutput(
                  (dense): Linear(in_features=768, out_features=768, bias=Tr

In [61]:
test_collator = TestCollator(0,1)
seq_length_torch = torch.tensor(seq_length,dtype=torch.long)
X_test_torch =torch.tensor(X_test, dtype=torch.long)
test = torch.utils.data.TensorDataset(X_test_torch,seq_length_torch)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False,collate_fn=test_collator)
test_preds = np.zeros((len(X_test)))

In [62]:
tk0 = tqdm_notebook(test_loader)
for i, x_batch in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [63]:
submission_bert_1 = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred
})
#submission_bert_1.to_csv('submission.csv', index=False)

In [64]:
submission_bert_1.head()

,id,prediction
0,7097320,0.000094
1,7097321,0.007118
2,7097322,0.070092
3,7097323,0.000373
4,7097324,0.000112


## BertModel_2 0.94154LB 0.94184CV

In [65]:
BERT_OUT = 128
DENSE_HIDDEN_UNITS = 128

In [66]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        #self.bert_layer = BertForSequenceClassification(config=bert_config,num_labels = BERT_OUT)
        self.bert_layer = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels = BERT_OUT)
        self.linear_out = nn.Linear(BERT_OUT, 1)
        self.linear_aux_out = nn.Linear(BERT_OUT, 6)
        self.drop_out_layer = nn.Dropout(p=0.1)        
        
    def forward(self, x, attention_mask=None, labels=None):
        bert_out = self.bert_layer(x, attention_mask=attention_mask, labels=labels)
        drop_out_layer_out = self.drop_out_layer(bert_out)
        result = self.linear_out(drop_out_layer_out)
        aux_result = self.linear_aux_out(bert_out)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [67]:
model = NeuralNet()
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load("../input/dense-128-260seq-reweightd-dropout1/dense_128_260seq_reweightd_dropout1_1.bin"))
model.cuda()
for param in model.parameters():
    param.requires_grad = False
model.eval()

DataParallel(
  (module): NeuralNet(
    (bert_layer): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1)
                )
                (output): BertSelfOutput(
                  (dense): Linear(in_features=768, out_features=768, bias=Tr

In [68]:
test_collator = TestCollator(0,1)
seq_length_torch = torch.tensor(seq_length,dtype=torch.long)
X_test_torch =torch.tensor(X_test, dtype=torch.long)
test = torch.utils.data.TensorDataset(X_test_torch,seq_length_torch)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False,collate_fn=test_collator)
test_preds = np.zeros((len(X_test)))

In [69]:
tk0 = tqdm_notebook(test_loader)
for i, x_batch in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [70]:
submission_bert_2 = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred
})
#submission_bert_2.to_csv('submission.csv', index=False)
submission_bert_2.head()

,id,prediction
0,7097320,0.000146
1,7097321,0.002046
2,7097322,0.164108
3,7097323,0.000535
4,7097324,0.000175


# Blending

In [71]:
pred = \
0.130*submission_lstm.prediction+\
0.350*submission_bert_large.prediction +\
0.175*submission_bert_1.prediction + 0.175*submission_bert_2.prediction + \
0.170*submission_gpt2_1.prediction

In [72]:
submission_blending = pd.DataFrame({
    "id":submission_bert_1.id,
    "prediction":pred
})
submission_blending.to_csv("submission.csv", index=False)

In [73]:
submission_blending.head(25)

,id,prediction
0,7097320,0.000195
1,7097321,0.010158
2,7097322,0.112465
3,7097323,0.002406
4,7097324,0.000555
5,7097325,0.000076
6,7097326,0.995423
7,7097327,0.327779
8,7097328,0.000142
9,7097329,0.002315
